# 🏥 SimuPaciente UMH - Demo en Google Colab

**Universidad Miguel Hernández de Elche**

Sistema de simulación de pacientes virtuales con voz para práctica de entrevistas clínicas.

---

## 📋 Instrucciones

1. **Ejecuta las celdas en orden** (Shift + Enter)
2. **Proporciona tu API key de OpenAI** cuando se te solicite
3. **Accede a la URL de ngrok** que se generará al final
4. **¡Comienza a practicar!**

⚠️ **IMPORTANTE:** Este notebook usa recursos de Colab gratuitos. La sesión se cerrará automáticamente después de ~12 horas de inactividad.

---

## 🔧 Paso 1: Instalación de Dependencias

Instalamos Python, Node.js y todas las dependencias necesarias.

In [ ]:
%%bash
echo "📦 Instalando dependencias del sistema..."

# Actualizar sistema
apt-get update -qq

# Instalar Node.js 18.x
curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
apt-get install -y nodejs

# Verificar versiones
echo "✅ Node.js version: $(node --version)"
echo "✅ npm version: $(npm --version)"
echo "✅ Python version: $(python3 --version)"

## 📥 Paso 2: Clonar Repositorio

Descargamos el código desde GitHub.

In [ ]:
import subprocess
import os

print("📥 Clonando repositorio desde GitHub...")

# Ir a /content primero
os.chdir('/content')

# Limpiar si existe
if os.path.exists('/content/ECOE'):
    subprocess.run(['rm', '-rf', 'ECOE'], check=True)

# Clonar repo
result = subprocess.run(
    ['git', 'clone', 'https://github.com/marcosbenghezala/ECOE.git', 'ECOE'],
    capture_output=True,
    text=True
)

if result.returncode != 0:
    print("❌ Error clonando repositorio:")
    print(result.stderr)
    raise RuntimeError("Failed to clone repository")

print("✅ Repositorio clonado exitosamente")

# Verificar que existe
if os.path.exists('/content/ECOE/simulador'):
    print("✅ Directorio /content/ECOE/simulador verificado")
    # Listar contenido
    result = subprocess.run(['ls', '-la', '/content/ECOE/'], capture_output=True, text=True)
    print(result.stdout)
else:
    print("❌ ERROR: El directorio no existe después de clonar")
    raise RuntimeError("Clone verification failed")

## 🐍 Paso 3: Instalar Dependencias Python

Instalamos las librerías de Python necesarias para el backend.

In [ ]:
%%bash
set -e  # Fallar si cualquier comando falla

# Verificar que el directorio existe
if [ ! -d "/content/ECOE/simulador" ]; then
    echo "❌ ERROR: El directorio /content/ECOE/simulador no existe"
    echo "⚠️  Asegúrate de haber ejecutado la celda de clonado (Paso 2) primero"
    exit 1
fi

cd /content/ECOE/simulador

echo "🐍 Instalando dependencias Python..."
pip install -q -r requirements.txt

echo "✅ Dependencias Python instaladas"

## ⚛️ Paso 4: Build del Frontend

Compilamos el frontend React.

In [ ]:
%%bash
set -e  # Fallar si cualquier comando falla

# Verificar que el directorio existe
if [ ! -d "/content/ECOE/simulador/frontend" ]; then
    echo "❌ ERROR: El directorio /content/ECOE/simulador/frontend no existe"
    echo "⚠️  Asegúrate de haber ejecutado la celda de clonado (Paso 2) primero"
    exit 1
fi

cd /content/ECOE/simulador/frontend

echo "⚛️ Instalando dependencias de Node.js..."
npm install --silent

echo "🔨 Building frontend..."
npm run build

echo "✅ Frontend compilado exitosamente"
ls -la /content/ECOE/simulador/frontend/dist/

## 🔑 Paso 5: Configurar API Key de OpenAI

**IMPORTANTE:** Ingresa tu API key de OpenAI de forma segura.

⚠️ **NUNCA compartas tu API key públicamente**

Obtén tu API key en: https://platform.openai.com/api-keys

In [ ]:
import os
from getpass import getpass

print("🔑 Configuración de API Key")
print("="*70)

# Solicitar OpenAI API Key de forma segura
openai_key = getpass("Ingresa tu OpenAI API Key: ")

if not openai_key.startswith('sk-'):
    print("⚠️ ADVERTENCIA: La API key no tiene el formato esperado (debe empezar con 'sk-')")
    print("Por favor verifica que hayas copiado la key completa.")
else:
    # Configurar variable de entorno
    os.environ['OPENAI_API_KEY'] = openai_key
    
    # Crear archivo .env
    with open('/content/ECOE/simulador/.env', 'w') as f:
        f.write(f"OPENAI_API_KEY={openai_key}\n")
    
    print("\n✅ API Key configurada exitosamente")
    print(f"✅ Key preview: {openai_key[:8]}...{openai_key[-4:]}")

## 🌐 Paso 6: Instalar y Configurar ngrok

ngrok nos permite exponer el servidor local a Internet de forma segura.

In [ ]:
%%bash
echo "🌐 Instalando Cloudflare Tunnel (cloudflared)..."

# Descargar cloudflared (alternativa a ngrok, sin autenticación)
wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
chmod +x cloudflared
mv cloudflared /usr/local/bin/

echo "✅ Cloudflare Tunnel instalado"
cloudflared --version

echo ""
echo "ℹ️  Usando Cloudflare Tunnel en lugar de ngrok"
echo "   - No requiere autenticación"
echo "   - Completamente gratuito"
echo "   - Sin límites de túneles"

## 🚀 Paso 7: Iniciar Servidor

**¡Última celda!** Esto iniciará el servidor y creará una URL pública.

⚠️ **IMPORTANTE:**
- Esta celda se ejecutará y terminará en ~40 segundos
- El servidor quedará corriendo en **background**
- Busca el botón azul con la URL pública
- La URL será tipo: https://xxx.trycloudflare.com

💡 **Nota:** Usamos Cloudflare Tunnel (gratuito, sin autenticación necesaria)

In [ ]:
import subprocess
import time
import requests
from IPython.display import display, HTML
import os
import re

print("🚀 Iniciando SimuPaciente UMH...")
print("="*70)

# Cambiar al directorio del simulador
os.chdir('/content/ECOE/simulador')

# PASO 1: Limpiar procesos anteriores CON FUERZA
print("\n🧹 Limpiando procesos anteriores...")
subprocess.run(['pkill', '-9', '-f', 'colab_server.py'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-9', '-f', 'cloudflared'], stderr=subprocess.DEVNULL)
subprocess.run(['fuser', '-k', '8080/tcp'], stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
time.sleep(3)

# Verificar que el puerto está libre
port_check = subprocess.run(['lsof', '-ti:8080'], capture_output=True, text=True)
if port_check.stdout.strip():
    print("⚠️ Puerto 8080 aún ocupado, forzando cierre...")
    pids = port_check.stdout.strip().split('\n')
    for pid in pids:
        subprocess.run(['kill', '-9', pid], stderr=subprocess.DEVNULL)
    time.sleep(2)
    print("✅ Puerto liberado")
else:
    print("✅ Puerto 8080 libre")

print("✅ Procesos limpiados")

# PASO 2: Verificar cloudflared
print("\n🔍 Verificando Cloudflare Tunnel...")
cf_check = subprocess.run(['which', 'cloudflared'], capture_output=True, text=True)
if not cf_check.stdout.strip():
    print("❌ cloudflared NO está instalado. Instalando ahora...")
    subprocess.run(['wget', '-q', 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64', '-O', '/tmp/cloudflared'], check=True)
    subprocess.run(['chmod', '+x', '/tmp/cloudflared'], check=True)
    subprocess.run(['mv', '/tmp/cloudflared', '/usr/local/bin/'], check=True)
    print("✅ cloudflared instalado")
else:
    print(f"✅ cloudflared encontrado")

# PASO 3: Iniciar servidor Flask en background
print("\n🏥 Iniciando servidor Flask...")
with open('/tmp/flask.log', 'w') as flask_log:
    server_process = subprocess.Popen(
        ['python3', 'colab_server.py'],
        stdout=flask_log,
        stderr=subprocess.STDOUT
    )

print("⏳ Esperando a que el servidor esté listo...")
time.sleep(12)

# Verificar que el servidor está corriendo
server_ok = False
for attempt in range(5):
    try:
        response = requests.get('http://localhost:8080/api/cases', timeout=5)
        print("✅ Servidor Flask iniciado correctamente")
        server_ok = True
        break
    except Exception as e:
        if attempt < 4:
            print(f"⏳ Intento {attempt + 1}/5...")
            time.sleep(3)

if not server_ok:
    print("\n❌ ERROR: El servidor no pudo iniciar correctamente")
    print("\n📋 Últimas 20 líneas del log:")
    print("-" * 70)
    with open('/tmp/flask.log', 'r') as f:
        lines = f.readlines()
        for line in lines[-20:]:
            print(line.rstrip())
    raise RuntimeError("Servidor Flask falló al iniciar. Revisa los logs arriba.")

# PASO 4: Iniciar Cloudflare Tunnel
print("\n🌐 Creando túnel público con Cloudflare...")
with open('/tmp/cloudflared.log', 'w') as cf_log:
    tunnel_process = subprocess.Popen(
        ['cloudflared', 'tunnel', '--url', 'http://localhost:8080'],
        stdout=cf_log,
        stderr=subprocess.STDOUT
    )

print("⏳ Esperando URL pública...")
time.sleep(8)

# PASO 5: Extraer URL del log de cloudflared
public_url = None
for attempt in range(10):
    try:
        with open('/tmp/cloudflared.log', 'r') as f:
            log_content = f.read()
            # Buscar URL en el formato: https://xxx.trycloudflare.com
            match = re.search(r'(https://[a-z0-9-]+\.trycloudflare\.com)', log_content)
            if match:
                public_url = match.group(1)
                print(f"✅ URL obtenida en intento {attempt + 1}")
                break
    except Exception as e:
        pass
    
    if attempt < 9:
        print(f"   Intento {attempt + 1}/10...")
        time.sleep(3)

if public_url:
    print("\n" + "="*70)
    print("✅ ✅ ✅  SERVIDOR LISTO  ✅ ✅ ✅")
    print("="*70)
    print(f"\n🌍 URL Pública: {public_url}")
    print("\n📋 Instrucciones:")
    print("   1. Haz click en el botón de abajo")
    print("   2. ¡Comienza a usar SimuPaciente!")
    print("\n⚠️  El servidor está corriendo en background")
    print("="*70)
    
    display(HTML(f'''
        <div style="
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            padding: 30px;
            border-radius: 15px;
            margin: 20px 0;
            text-align: center;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        ">
            <h2 style="color: white; margin-bottom: 20px; font-size: 24px;">🏥 SimuPaciente UMH</h2>
            <a href="{public_url}" target="_blank" style="
                display: inline-block;
                background: white;
                color: #667eea;
                padding: 15px 40px;
                text-decoration: none;
                border-radius: 25px;
                font-weight: bold;
                font-size: 18px;
                box-shadow: 0 5px 15px rgba(0,0,0,0.2);
            ">
                🚀 Abrir Aplicación
            </a>
            <p style="color: white; margin-top: 20px; font-size: 14px;">
                {public_url}
            </p>
            <p style="color: white; margin-top: 10px; font-size: 12px; opacity: 0.8;">
                Powered by Cloudflare Tunnel
            </p>
        </div>
    '''))
    
    print("\n✅ Celda ejecutada exitosamente")
    print("📊 Logs disponibles:")
    print("   - Servidor: /tmp/flask.log")
    print("   - Túnel: /tmp/cloudflared.log")
    
else:
    print("\n" + "="*70)
    print("❌ No se pudo obtener la URL pública")
    print("="*70)
    
    # Leer logs de cloudflared
    print("\n📋 Últimas 30 líneas del log de cloudflared:")
    print("-" * 70)
    try:
        with open('/tmp/cloudflared.log', 'r') as f:
            lines = f.readlines()
            for line in lines[-30:]:
                print(line.rstrip())
    except Exception as e:
        print(f"⚠️ No se pudo leer el log: {e}")
    
    print("\n" + "="*70)
    print("\n💡 Soluciones:")
    print("   1. Ejecuta esta celda nuevamente")
    print("   2. Verifica tu conexión a Internet")
    print("="*70)

---

## 📝 Notas Adicionales

### 🔄 Reiniciar el Servidor
Si necesitas reiniciar:
1. Detén la celda anterior (botón Stop)
2. Ejecuta la celda nuevamente

### 🐛 Solución de Problemas

**Problema:** No se conecta a OpenAI Realtime API
- Verifica que tu API key tiene acceso a Realtime API (beta)
- Comprueba que tienes créditos disponibles

**Problema:** La URL de ngrok no funciona
- Espera 30 segundos y recarga la página
- Verifica que la celda del servidor sigue corriendo [*]

### 📧 Soporte
- GitHub: https://github.com/marcosbenghezala/ECOE
- Universidad Miguel Hernández de Elche

---

**Versión:** 0.9.5 (Release Candidate)  
**Última actualización:** 17 de diciembre de 2025